In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris.data
X

In [ ]:
y = iris.target
y

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
X = scaler_x.fit_transform(X)
X

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categorical_features = [0])
y.shape

In [ ]:
y = y.reshape(-1, 1)
y.shape

In [ ]:
y = onehot.fit_transform(y).toarray()
y

In [ ]:
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.3)

In [ ]:
X_treinamento.shape

In [ ]:
X_teste.shape

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
neuronios_entrada = X.shape[1]
neuronios_entrada

In [ ]:
neuronios_oculta = int(np.ceil((X.shape[1] + y.shape[1]) / 2))
neuronios_oculta

In [ ]:
neuronios_saida = y.shape[1]
neuronios_saida

In [ ]:
W = {'oculta': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta, neuronios_saida]))}

In [ ]:
b = {'oculta': tf.Variable(tf.random_normal([neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}

In [ ]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [ ]:
def mlp(x, W, bias):
    camada_oculta = tf.add(tf.matmul(x, W['oculta']), bias['oculta'])
    camada_oculta_ativacao = tf.nn.relu(camada_oculta)
    camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, W['saida']), b['saida'])
    return camada_saida

In [ ]:
modelo = mlp(xph, W, b)

In [ ]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(erro)

In [ ]:
batch_size = 8
batch_total = int(len(X_treinamento) / batch_size)
batch_total

In [ ]:
X_batches = np.array_split(X_treinamento, batch_total)
X_batches

In [ ]:
X_batches[1]

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(3000):
        erro_medio = 0.0
        batch_total = int(len(X_treinamento) / batch_size)
        X_batches = np.array_split(X_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            X_batch, y_batch = X_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: X_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 500 == 0:
            print('Época: ' + str((epoca + 1)) + ' erro: ' + str(erro_medio))
    W_final, b_final = sess.run([W, b])

In [ ]:
W_final

In [ ]:
b_final

In [ ]:
# previsões
previsoes_teste = mlp(xph, W_final, b_final)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    r1 = sess.run(previsoes_teste, feed_dict = {xph: X_teste})
    r2 = sess.run(tf.nn.softmax(r1))
    r3 = sess.run(tf.argmax(r2, 1))

In [ ]:
r1

In [ ]:
r2

In [ ]:
r3

In [ ]:
y_teste2 = np.argmax(y_teste, 1)
y_teste2

In [ ]:
from sklearn.metrics import accuracy_score
taxa_acerto = accuracy_score(y_teste2, r3)
taxa_acerto